In [1]:
# Define Imports
import os
import googlemaps
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc
import pydeck as pdk
from dash import Dash, dcc, html
from dotenv import load_dotenv

In [2]:
# Read and explore raw data
df = pd.read_csv("../../data/submissions.csv")
df.head()

,submission_code,submission_date,submission_status,submission_description,initiator_id,initiator_legalname,initiator_address,initiator_region,item_amount,item_classification
0,010624-040A8668,2024-06-01T17:36:27.658298Z,pending,Харчоблок базової кухні,4394869,ВЕЛИКОБІРКІВСЬКА СЕЛИЩНА РАДА,"Тернопільська обл., Тернопільський р-н, селище...",Тернопільська,3967203.0,kitchen
1,020624-2C34FAB7,2024-06-02T16:32:18.130273Z,pending,харчоблок,2146222,"УПРАВЛІННЯ ОСВІТИ, МОЛОДІ ТА СПОРТУ ЛОЗІВСЬКОЇ...","Харківська обл., місто Лозова, МІКРОРАЙОН 1, б...",Харківська,9607874.0,kitchen
2,020624-9F163A6A,2024-06-02T18:06:32.388695Z,pending,харчоблок,2146222,"УПРАВЛІННЯ ОСВІТИ, МОЛОДІ ТА СПОРТУ ЛОЗІВСЬКОЇ...","Харківська обл., місто Лозова, МІКРОРАЙОН 1, б...",Харківська,16569763.0,kitchen
3,020624-D8610EBB,2024-06-02T17:09:51.508281Z,pending,харчоблок,2146222,"УПРАВЛІННЯ ОСВІТИ, МОЛОДІ ТА СПОРТУ ЛОЗІВСЬКОЇ...","Харківська обл., місто Лозова, МІКРОРАЙОН 1, б...",Харківська,20051019.0,kitchen
4,020624-E1B2F3B0,2024-06-02T16:20:55.533024Z,pending,харчоблок,2146222,"УПРАВЛІННЯ ОСВІТИ, МОЛОДІ ТА СПОРТУ ЛОЗІВСЬКОЇ...","Харківська обл., місто Лозова, МІКРОРАЙОН 1, б...",Харківська,16166182.0,kitchen


In [3]:
# Load ENV variables
load_dotenv(dotenv_path='../../.env')

True

In [4]:
# Get the Google Maps API key from the environment variables
gmaps_api_key = os.getenv('GMAPS_API_KEY')

# Initialize the Google Maps client with the API key
gmaps = googlemaps.Client(key=gmaps_api_key)

# Function to geocode an address
def geocode_address(address):
    try:
        geocode_result = gmaps.geocode(address)
        if geocode_result:
            location = geocode_result[0]['geometry']['location']
            return location['lat'], location['lng']
        else:
            return None, None
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
        return None, None

In [5]:
# Geocode Regions in Dataset to have Lat and Long
regions = pd.DataFrame(df.groupby('initiator_region', as_index=False).first()['initiator_region'])
regions[['latitude', 'longitude']] = regions['initiator_region'].apply(lambda x: pd.Series(geocode_address(x)))
regions

,initiator_region,latitude,longitude
0,Івано-Франківська,48.920062,24.708916
1,Волинська,50.747233,25.325383
2,Вінницька,49.233083,28.468217
3,Дніпропетровська,48.464717,35.046183
4,Житомирська,50.254650,28.658667
5,Закарпатська,48.620800,22.287883
6,Запорізька,47.838800,35.139567
7,Київська,50.052951,30.766713
8,Кіровоградська,48.507933,32.262317
9,Львівська,49.839683,24.029717


In [ ]:
# Save geocoded Regions
regions.to_csv("../../data/regions_geocoded.csv", index=False)